<a href="https://colab.research.google.com/github/kocielnik/rule_one_stocks/blob/main/rule_one_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom functions

In [ ]:
%%file deals.py

"""
Sue is a Rule One Investor.

She wants to check stock prices daily to see if any Rule One deals are available
for her.

For simplicity and robustness of examples, the `Ticker` class in these examples
is replaced with MockTicker.

When Sue uses this program, she always omits the part `Ticker=MockTicker`.


One day, Sue wants to check the price of a single stock.

>>> company = Company(symbol="tsm", sticker_price=11)
>>> get_deal(company, Ticker=MockTicker)
Deal(symbol='tsm', sticker_price=11, price=98, percent_of_sticker=891)


On another day, Sue has several companies she wants to check the price of.

To the same method `get_deal`, she can pass a list of companies the same way
she passed a single company.

>>> companies = [
...  Company(symbol="msft", sticker_price=118),
...  Company(symbol="tsm", sticker_price=11)
... ]
>>> deals = get_deal(companies, Ticker=MockTicker)
>>> deals[0]
Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
>>> deals[1]
Deal(symbol='tsm', sticker_price=11, price=98, percent_of_sticker=891)


Sue wants the results to be sorted in the order of the best deal available.

Best deal means the largest Margin of Safety (MOS).

If companies are not sorted in the order of the best deal, the results *should*
be.

Note the order of input companies is reversed with respect to the previous example.

>>> companies = [
...  Company(symbol="tsm", sticker_price=11),
...  Company(symbol="msft", sticker_price=118)
... ]
>>> deals = get_deal(companies, Ticker=MockTicker)
>>> deals[0]
Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)


Sue says the interface is too complicated for her, and she would love to
be able to initialize Deal instances directly from Company instances.

>>> companies = [
...  Company(symbol="tsm", sticker_price=11),
...  Company(symbol="msft", sticker_price=118)
... ]
>>> deals = Deal.from_(companies, Ticker=MockTicker)
>>> deals[0]
Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
"""

from typing import NamedTuple
from yfinance import Ticker
from requests import get
from functools import lru_cache


class Company(NamedTuple):
    symbol: str
    sticker_price: float


class Deal(NamedTuple):

    symbol: str
    sticker_price: float
    price: float
    percent_of_sticker: int

    """
    Sue wants to print an instance of this class and be able to copy-paste
    it into another place, or save it for later pasting.

    >>> Deal(symbol='msft', sticker_price=1, price=1)
    Deal(symbol='msft', sticker_price=1, price=1)


    Sue wants to be able to compare instances of `Deal` to check for equality,
    in order to remove duplicates from lists if any are found.

    She does not remember, *why* exactly she wanted this feature. Still, she
    insists it should be available.

    >>> deal_1 = Deal(symbol='msft', sticker_price=1, price=1)
    >>> deal_2 = Deal(symbol='msft', sticker_price=1, price=1)
    >>> deal_1 == deal_2
    True
    """

    def __eq__(self, other):
        if repr(other) != repr(self):
            return False
        return True

    def __lt__(self, other):
        return self.percent_of_sticker < other.percent_of_sticker

    @staticmethod
    def from_(company, Ticker=Ticker):
        deal = get_deal(company, Ticker=Ticker)
        return deal


class MockTicker:
    def __init__(self, symbol):
        price = 352.5 if symbol == "msft" else 98

        self.info = {
            "currentPrice": price
        }


def get_price(symbol, Ticker=Ticker):

    """
    >>> get_price("msft", Ticker=MockTicker)
    352.5
    >>> get_price([])
    {}
    >>> get_price(["msft"], Ticker=MockTicker)
    {'msft': 352.5}
    """

    if type(symbol) == str:
        price = Ticker(symbol).info["currentPrice"]
        return price

    prices = {
        the_symbol: get_price(the_symbol, Ticker=Ticker)
        for the_symbol in symbol
    }
    return prices

def get_percent_of_sticker(price, sticker_price):
    return int(100 * round(price/sticker_price, 2))

def get_deal(company, Ticker=Ticker):

    """
    >>> company = Company(symbol="msft", sticker_price=118)
    >>> deal = get_deal(company, Ticker=MockTicker)
    >>> deal
    Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
    """

    if isinstance(company, list):
        return get_deals(company, Ticker=Ticker)

    price = get_price(company.symbol, Ticker=Ticker)
    return Deal(
        symbol=company.symbol,
        sticker_price=company.sticker_price,
        price=price,
        percent_of_sticker=get_percent_of_sticker(price, company.sticker_price)
    )

def get_deals(companies, Ticker=Ticker):

    """
    >>> companies = [
    ...   Company("msft", 118),
    ...   Company("tsm", 22)
    ... ]
    >>> deals = get_deals(companies, Ticker=MockTicker)
    >>> deals[0]
    Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
    """

    sticker_prices = {
        company.symbol: company.sticker_price
        for company in companies
    }

    deals = []
    for symbol in sticker_prices.keys():
        price = get_price(symbol, Ticker=Ticker)
        sticker_price=sticker_prices[symbol]
        deals.append(
            Deal(symbol=symbol,
                 sticker_price=sticker_price,
                 price=price,
                 percent_of_sticker=get_percent_of_sticker(price, sticker_price)
            )
        )

    return sorted(deals)

def mock_get_sticker_price(symbol):
    class MockResponse:
        def json():
            return {"sticker_price": {"value": 22}}

    return MockResponse

@lru_cache
def get_sticker(symbol, api_host="143.42.16.225:8080", get=get):
    """
    >>> round(get_sticker("tsm", get=mock_get_sticker_price))
    22
    """
    url = f"http://{api_host}/search/{symbol}"
    result = get(url).json()["sticker_price"]["value"]
    return result

def round_sticker(sticker):

    """
    Prices above $2 should be rounded to the nearest integer.
    >>> round_sticker(22.01)
    22

    Prices below $2 should be rounded to full cents.
    >>> round_sticker(0.512)
    0.51
    """
    return round(sticker) if sticker >= 2 else round(sticker, 2)

def is_valid_sticker(result):
    return result is not None and result >= 0

def get_stickers(items):
    stickers = {}
    for key in items.keys():
        result = get_sticker(key)
        if is_valid_sticker(result):
            stickers[key] = round_sticker(result)
    return stickers

def with_preview(value):
    print(value)
    return value


class StickerPriceIngredients(NamedTuple):

    """
    >>> inputs = StickerPriceIngredients(
    ...     current_eps=1.31,
    ...     future_growth_rate_percent=15,
    ...     future_pe=30,
    ...     minimum_acceptable_rate_of_return_percent=15,
    ...     years=10
    ... )
    """

    current_eps: float
    future_growth_rate_percent: float
    future_pe: float
    minimum_acceptable_rate_of_return_percent: float
    years: int



class StickerPriceResults(NamedTuple):

    """
    Compare https://www.ruleoneinvesting.com/margin-of-safety-calculator/

    >>> outputs = StickerPriceResults(
    ...     future_eps=11,
    ...     future_value=254,
    ...     sticker_price=63,
    ...     mosp=31
    ... )
    >>> outputs.future_eps
    11
    """

    future_eps: float
    future_value: float
    sticker_price: float
    mosp: float

def get_sticker_price():

    """
    Expected results - not attained yet:

    >>> get_sticker_price().future_eps
    10.48

    Should be 11.

    >>> get_sticker_price().future_value
    314

    Should be 254.
    """

    ingredients = StickerPriceIngredients(
        current_eps=1.31,
        future_growth_rate_percent=15,
        future_pe=30,
        minimum_acceptable_rate_of_return_percent=15,
        years=10
    )

    years_to_double = 72 // ingredients.future_growth_rate_percent
    num_doubles = ingredients.years // years_to_double
    future_eps = 2 ** (1 + num_doubles) * ingredients.current_eps

    future_value = round(future_eps*ingredients.future_pe)

    results = StickerPriceResults(
        future_eps=future_eps,
        future_value=future_value,
        sticker_price=0,
        mosp=0
    )

    return results

Writing deals.py


### Tests

In [ ]:
!pytest --doctest-modules deals.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.2.2, pluggy-1.2.0
rootdir: /content
plugins: anyio-3.7.1
collected 9 items                                                              

deals.py .........                                                       [100%]

============================== 9 passed in 0.63s ===============================


In [ ]:
# symbol: sticker_price
watch_list_items = {'TSM': 22,
 'ON': 24,
 'TSLA': 54,
 'MSFT': 118,
 'APH': 23,
 'LSCC': 0.5,
 'ACN': 110,
 'GOOG': 93,
 'ADBE': 118,
 'ADI': 37,
 'ANSS': 57,
 'AMAT': 72,
 'MELI': 2500,
 'MCHP': 44,
 'TSCO': 88,
 'SNPS': 50,
 'CPRT': 142,
 'UMC': 0.97,
 'ARCB': 75,
 'MBUU': 98,
 'PERI': 66,
 'NVR': 721,
 'WSO': 51,
 'WEEV': None,
 'UNH': 350,
 'HOG': 18,
 'ULTA': 343,
 'TW': 84,
 'CMG': 201,
 'ADSK': 240,
 'DE': 435,
 'LSTR': 145,
 'MU': None,
 'NFLX': 691,
 'ABNB': None}

## Update sticker prices if available

In [ ]:
from deals import with_preview, get_stickers

stickers = with_preview(get_stickers(watch_list_items))

{}


In [ ]:
watch_list_items.update(stickers)
watch_list_items

{'TSM': 22,
 'ON': 24,
 'TSLA': 54,
 'MSFT': 118,
 'APH': 23,
 'LSCC': 0.5,
 'ACN': 110,
 'GOOG': 93,
 'ADBE': 118,
 'ADI': 37,
 'ANSS': 57,
 'AMAT': 72,
 'MELI': 2500,
 'MCHP': 44,
 'TSCO': 88,
 'SNPS': 50,
 'CPRT': 142,
 'UMC': 0.97,
 'ARCB': 75,
 'MBUU': 98,
 'PERI': 66,
 'NVR': 721,
 'WSO': 51,
 'WEEV': None,
 'UNH': 350,
 'HOG': 18,
 'ULTA': 343,
 'TW': 84,
 'CMG': 201,
 'ADSK': 240,
 'DE': 435,
 'LSTR': 145,
 'MU': None,
 'NFLX': 691}

## Example

In [ ]:
# Import classes, reload if needed.
from importlib import reload; import deals; reload(deals)
from deals import Company, Deal

#
companies = [
    Company(symbol=symbol, sticker_price=watch_list_items[symbol])
    for symbol in watch_list_items.keys()
    if watch_list_items[symbol] is not None
]

deals = Deal.from_(companies)
deals

[Deal(symbol='PERI', sticker_price=66, price=35.07, percent_of_sticker=53),
 Deal(symbol='MELI', sticker_price=2500, price=1387.06, percent_of_sticker=55),
 Deal(symbol='MBUU', sticker_price=98, price=56.65, percent_of_sticker=57),
 Deal(symbol='CPRT', sticker_price=142, price=87.01, percent_of_sticker=61),
 Deal(symbol='NFLX', sticker_price=691, price=429.98, percent_of_sticker=62),
 Deal(symbol='ADSK', sticker_price=240, price=208.06, percent_of_sticker=87),
 Deal(symbol='TW', sticker_price=84, price=81.5, percent_of_sticker=97),
 Deal(symbol='DE', sticker_price=435, price=434.97, percent_of_sticker=100),
 Deal(symbol='ULTA', sticker_price=343, price=447.46, percent_of_sticker=130),
 Deal(symbol='LSTR', sticker_price=145, price=198.4, percent_of_sticker=137),
 Deal(symbol='GOOG', sticker_price=93, price=130.21, percent_of_sticker=140),
 Deal(symbol='UNH', sticker_price=350, price=503.39, percent_of_sticker=144),
 Deal(symbol='ARCB', sticker_price=75, price=109.43, percent_of_sticker=

## Fixing Sticker Price

In [ ]:
!pip install --quiet requests_futures

In [ ]:
!rm -rf IsThisStockGood && git clone --quiet https://github.com/kocielnik/IsThisStockGood.git

In [ ]:
import sys
sys.path.append("IsThisStockGood")

from src.DataFetcher import fetchDataForTickerSymbol

data = fetchDataForTickerSymbol("MELI")
data

{'ticker': 'MELI',
 'name': 'null',
 'roic': [31.96, 17.24, -33.89, 3.95],
 'eps': [473.05, 66.06, 98.57, 15.29],
 'sales': [49.06, 66.17, 54.0, 41.19],
 'equity': [17.97, -5.32, 37.52, 18.66],
 'cash': [598.03, 99.23, 66.51, 64.42],
 'total_debt': 5897000000.0,
 'free_cash_flow': 2485000000.0,
 'debt_payoff_time': 2.3730382293762577,
 'debt_equity_ratio': 2.1168,
 'margin_of_safety_price': 'null',
 'current_price': 'null',
 'sticker_price': 'null',
 'payback_time': 'null',
 'average_volume': 'null'}